# 3. Issue Dianosis: Agent Bricks Custom LLM

Data Flow: raw reviews -> review aspect extractions -> location aspect daily -> flag all issues -> **issue diagnosis and recommendations**


## Build Custom LLM Agent (Summary + Diagnosis)

- Instructions: 
```
You are an experienced hotel operations analyst.
Given a single issue aspect (for example, “cleanliness” or “temperature_hvac”) and several short review extracts related to that issue, write a concise and factual diagnosis summarizing what the data indicates.

Rules:
	1.	Base everything strictly on the provided reviews. Identify common complaints and patterns across guests.
	2.	Do not make up facts — only infer root causes that clearly align with evidence.
	3.	Keep tone factual, professional, and concise.
	4.	Summarize in the following four parts only:
		• issue_summary → what the guests are saying (pattern)
		• potential_root_cause → why it’s likely happening
		• impact → how it affects guest experience or brand
		• recommended_action → clear, actionable steps to fix or prevent recurrence
	5.	The output must be valid JSON. No markdown, no additional commentary.
	6.	Keep total text per field under ~3 sentences.

Output format:
{
  "aspect": "<aspect_name>",
  "issue_summary": "<short summary of the pattern in guest reviews>",
  "potential_root_cause": "<likely operational or maintenance causes>",
  "impact": "<how it affects guest satisfaction, comfort, or reputation>",
  "recommended_action": "<specific and practical steps to address the issue>"
}
```

## Batch Inference With CLLM Agent Endpoint and `AI_QUERY`
[CLLM Query in SQL Editor](queries/AI Query with CLLM Diagnosis.dbquery.ipynb)

In [0]:
%sql
CREATE OR REPLACE TABLE lakehouse_inn_catalog.voc.open_issues_diagnosis AS
WITH query_results AS (
  SELECT
    *,
    ai_query(
      't2t-e71a8879-endpoint',
      relevant_reviews_text,
      failOnError => false
    ) AS response
  FROM (
    SELECT *
    FROM lakehouse_inn_catalog.voc.issues_daily
    -- WHERE status = 'Open'
  )
)
SELECT
  -- Exclude the response column
  issue_id,
  opened_at,
  location,
  aspect,
  severity,
  status,
  open_reason,
  delta_pp_open,
  nms_open,
  volume_open,
  relevant_reviews,
  response.result AS response_result,
  response.errorMessage AS response_error,
  current_date() AS processed_date
FROM query_results;



In [0]:
display(spark.table('lakehouse_inn_catalog.voc.open_issues_diagnosis'))

## Next Step: AI/BI Dashboard & Genie Room